In [1]:
from utils.Data_by_API import * 

In [8]:
import pandas as pd
from bs4 import BeautifulSoup
from collections import defaultdict
from utils.Data_by_API import *

class Bus_Stop_Data_by_API(Data_by_API):
    
    base_url = "http://openapi.its.ulsan.kr/UlsanAPI/RouteInfo.xo?"
    
    def __init__(self, params_dict):            
        super().__init__(url = self.base_url)
        self.request_url = super().create_request_url(params_dict = params_dict)
#         self.year = year
        self.params_dict = params_dict
        self.type = "xml"
        
    def extract_values_from_dict(self, dct):
        dict_list = dct["tableInfo"]["list"]["row"]
            
        return dict_list
    
    
    def calculate_max_page(self, type = "json"):
        rq = self.request()
        rq_dict = self.to_dict(txt = rq.text, type = type)
        
        self.n_rows = int(self.params_dict["numOfRows"])
        
        try:
            self.total_count = int(rq_dict["response"]["body"]["totalCount"])
        except:
            xmlsoup = BeautifulSoup(rq.text,'html.parser')
            self.total_count = int(xmlsoup.find('totalcnt').text)
                
        max_page = int(np.ceil(self.total_count / self.n_rows))
        
        print(f"n_rows : {self.n_rows}, total_count : {self.total_count}, max_page = {max_page}")
        
        return max_page
    
    
    def get(self):
        self.request_urls = self.create_request_urls()
        
        data_dict = defaultdict(list)
        for request_url in self.request_urls:
            
            rq = super().request(request_url = request_url)
            temp_dict = super().parse(request = rq, features = None, type = self.type)
            
            for k, v in temp_dict.items():
                data_dict[k].extend(v)

                    
        return pd.DataFrame(data_dict)

In [9]:
serviceKey = "eLWdQyzctRdtv8bEOuewsTtK6sNkoWp1bE74OUBk43jg4tU6AsI6yYt6Z%2B7sOeaqtB5pTH2yHuPRIuEHtu5amQ%3D%3D"

In [10]:
params_dict = {"serviceKey" : serviceKey, 
               "pageNo" : "1",
               "numOfRows" : "1000"}

In [11]:
bus_stop = Bus_Stop_Data_by_API(params_dict = params_dict)

In [12]:
bus_stop.get()

n_rows : 1000, total_count : 484, max_page = 1


,DIRECTION,COMPANY,BRTNAME,BRTTYPE,CLASS,STOPEDID,RNUM,BRTNO,STOPSTID,BRTID
0,2,약사마을,10(성안청구 방면),20,0,192011701,1,10,192062002,192100102
1,1,None,10(풍암마을 방면),20,0,192062001,2,10,192072202,192100101
2,1,우리버스,10(풍암마을 방면),20,1,192062001,3,10,192022703,192100103
3,3,울산여객(주),101(꽃바위순환),13,0,194019016,4,101,194019014,194101013
4,3,울산여객(주),101(꽃바위순환),13,2,194019016,5,101,194019014,194101015
...,...,...,...,...,...,...,...,...,...,...
479,3,우리버스,977(수필아파트 순환),30,2,193065004,480,977,193076401,193109775
480,3,우리버스,977(수필아파트 순환),30,1,193040749,481,977,193065004,193109774
481,3,울남지선버스(주)농소지점,995(박상진역사공원 방면),30,2,195040360,482,995,195071411,195109955
482,3,울남지선버스(주)농소지점,995(오토밸리줌파크 방면),30,1,195071411,483,995,195040363,195109954
